In [1]:
import json
import pandas as pd 
import numpy as np
from pprint import pprint
import warnings 
warnings.filterwarnings('ignore')
import datetime
from dateutil import relativedelta
import re

In [2]:
population_data= pd.read_excel('DDW_PCA0000_2011_Indiastatedist.xlsx',usecols=['Level','Name','TRU','TOT_P'])
# population_data.head()

In [3]:
census_data_state = population_data.loc[(population_data['Level'] == 'STATE') & (population_data['TRU'] == 'Total')]
census_data_state = census_data_state.drop(columns=["TRU"])
census_data_state["Name"] = census_data_state["Name"].str.lower()
census_data_state['TOT_P'] = census_data_state['TOT_P'].astype(float)
census_data_state = census_data_state.sort_values('Name',ignore_index=True)
# len(census_data_district) is 640
census_data_state1 = census_data_state.copy()
# census_data_state

#### Renaming some states accoring to vaccine data 

In [4]:
census_data_state1.iat[0,1] = 'andaman and nicobar islands'
census_data_state1.iat[24,1] = 'delhi'
census_data_state1 = census_data_state1.sort_values('Name',ignore_index=True)
# census_data_state1

In [5]:
# vaccine_data1

In [6]:
census_data_state1.iat[7,2] += census_data_state1.iat[8,2]
census_data_state1.drop([8], inplace=True)
census_data_state1 = census_data_state1.reset_index()
census_data_state1 = census_data_state1.drop(columns=["index"])
census_data_state1.iat[7,1] = 'dadra and nagar haveli and daman and diu'
census_data_state1.iat[13,1] =  'jammu and kashmir'
# census_data_state1

In [7]:
# census_data_state1

In [8]:
vaccine_data1 = pd.read_csv("cowin_vaccine_data_districtwise.csv",usecols=['State_Code','State','31/08/2021.3'])
vaccine_data1.rename(columns = {'31/08/2021.3':'First Dose Administered'}, inplace = True)
vaccine_data1 = vaccine_data1.drop(vaccine_data1.index[0])
## Making all the district names into small letters so that merging will be easy ##
vaccine_data1["State"] = vaccine_data1["State"].str.lower()

# Changing vaccination numbers values type from object to numeric ##
vaccine_data1['First Dose Administered'] = vaccine_data1['First Dose Administered'].astype(float)

# Sorting vaccination data on the basic of State values ##
vaccine_data1 = vaccine_data1.groupby(['State_Code','State']).agg({'First Dose Administered':'sum'}).reset_index()  
vaccine_data1 = vaccine_data1.sort_values('State',ignore_index=True)
# vaccine_data1

In [9]:
vaccine_data = vaccine_data1.copy()
vaccine_data.iat[13,2] += vaccine_data.iat[17,2]
vaccine_data.iat[30,2] += vaccine_data.iat[31,2]
vaccine_data.drop([17], inplace=True)
vaccine_data.drop([31], inplace=True)
# vaccine_data.at[14,'First Dose Administered'] = 0
vaccine_data = vaccine_data.sort_values('State',ignore_index=True)
# vaccine_data

In [10]:
# census_data_state1.head()

In [11]:
census_data_state1['Name'] = census_data_state1['Name'].str.strip()
vaccine_data['State'] = vaccine_data['State'].str.strip()
data_sample = pd.merge(census_data_state1,vaccine_data,left_on='Name', right_on='State')
data_sample = data_sample.drop(columns=["Level","Name","State"])
data_sample = data_sample[['State_Code','First Dose Administered','TOT_P']]
data_sample['Populatio_not_vaccinated'] = data_sample['TOT_P'] - data_sample['First Dose Administered']
# data_sample

In [12]:
rate_original = pd.read_csv("cowin_vaccine_data_districtwise.csv",usecols=['State_Code','State','14/08/2021.3','07/08/2021.3'])
rate_original.rename(columns = {'14/08/2021.3':'Dose_Administered_14aug','07/08/2021.3':'Dose_Administered_07aug'}, inplace = True)
rate_original = rate_original.drop(rate_original.index[0])
rate_original = rate_original.drop(columns=["State"])

rate_original['Dose_Administered_07aug'] = rate_original['Dose_Administered_07aug'].astype(float)
rate_original['Dose_Administered_14aug'] = rate_original['Dose_Administered_14aug'].astype(float)

rate_original = rate_original.groupby(['State_Code']).agg({'Dose_Administered_07aug':'sum',
                                       'Dose_Administered_14aug':'sum'}).reset_index()  
# rate_original

In [13]:
rate_copy = rate_original.copy()
rate_copy.iat[14,2] += rate_copy.iat[17,2]
rate_copy.iat[31,2] += rate_copy.iat[30,2]
rate_copy.drop([17], inplace=True)
rate_copy.drop([30], inplace=True)
rate_copy = rate_copy.sort_values('State_Code',ignore_index=True)
rate_copy['Dose_Administered_LastWeek'] = rate_copy['Dose_Administered_14aug'] - rate_copy['Dose_Administered_07aug']
rate_copy = rate_copy.drop(columns=["Dose_Administered_07aug","Dose_Administered_14aug"])
# rate_copy.head()

In [14]:
rate_copy['Dose_Administered_LastWeek'] = rate_copy['Dose_Administered_LastWeek'] / 7
# rate_copy.head()

In [15]:
pre_final = pd.merge(data_sample,rate_copy,left_on='State_Code', right_on='State_Code')
pre_final.rename(columns = {'Dose_Administered_LastWeek':'Vaccination_Rate_LastWeek'}, inplace = True)
# pre_final.head()

In [21]:
pre_final = pre_final.round(3)
pre_final

,State_Code,First Dose Administered,TOT_P,Populatio_not_vaccinated,Vaccination_Rate_LastWeek
0,AN,257452.0,380581.0,123129.0,2526.857
1,AP,21981121.0,84580777.0,62599656.0,148431.143
2,AR,726441.0,1383727.0,657286.0,1850.571
3,AS,14421757.0,31205576.0,16783819.0,177262.286
4,BR,31666711.0,104099452.0,72432741.0,317408.429
5,CH,807067.0,1055450.0,248383.0,6710.000
6,CT,10007608.0,25545198.0,15537590.0,37209.571
7,DN,617182.0,586956.0,-30226.0,2584.714
8,DL,9657498.0,16787941.0,7130443.0,72391.571
9,GA,1166210.0,1458545.0,292335.0,3632.857


In [17]:
def get_date(count,rate):
    date_str = '31/08/2021'
    a = np.ceil(count/rate)
    date = datetime.datetime.strptime(date_str, '%d/%m/%Y')
    date = date + datetime.timedelta(days=a)
    return return_date(date)


In [18]:
def return_date(date_case):
    date_case = str(date_case)
    match_str = re.search(r'\d{4}-\d{2}-\d{2}', date_case)
    res = datetime.datetime.strptime(match_str.group(), '%Y-%m-%d').date()
    return str(res)

In [19]:
Final_data = pd.DataFrame()
list = ['stateid', 'populationleft', 'rateofvaccination', 'date']
temp_final = pd.DataFrame(columns=list)
for i in pre_final.index:
    temp_final['stateid'] = [pre_final.iat[i,0]]
    temp_final['populationleft'] = [pre_final.iat[i,3]]
    temp_final['rateofvaccination'] = [pre_final.iat[i,4]]
    temp_final['date'] = [get_date(pre_final.iat[i,3],pre_final.iat[i,4])]
    Final_data = Final_data.append(temp_final, ignore_index=True)
Final_data

,stateid,populationleft,rateofvaccination,date
0,AN,123129.0,2526.857,2021-10-19
1,AP,62599656.0,148431.143,2022-10-27
2,AR,657286.0,1850.571,2022-08-22
3,AS,16783819.0,177262.286,2021-12-04
4,BR,72432741.0,317408.429,2022-04-17
5,CH,248383.0,6710.000,2021-10-08
6,CT,15537590.0,37209.571,2022-10-23
7,DN,-30226.0,2584.714,2021-08-20
8,DL,7130443.0,72391.571,2021-12-08
9,GA,292335.0,3632.857,2021-11-20


In [20]:
Final_data.to_csv('complete_vaccination.csv', index=False)

# Q9 Completed..:)